In [1]:
import pandas as pd
from utils import train_test_indexes
from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn.preprocessing import StandardScaler
from randForest_model import SklearnClfModel
from hyperopt import hp
import numpy as np
from apply_clf_model import find_label
from joblib import Parallel, delayed
import multiprocessing
from scipy import io

%matplotlib inline
import mpld3
#mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
train = io.loadmat('/home/max/projects/challengeMDI343/data/data_train.mat')

gallery = {train['galleryId'].ravel()[i]: train['galleryLabel'].ravel()[i] for i in range(len(train['galleryLabel']))}
probes = { train['probeId'].ravel()[i]: train['probeLabel'].ravel()[i] for i in range(len(train['probeLabel']))}

In [3]:
df = pd.read_pickle('data/formated_train_data_7l.pkl')

print df.shape
df.head(20)

(2652328, 13)


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim
0,2732.521729,0.000000,0.000000,0,0,0,0,0,41004,13721,7490,13721,1
1,2544.442139,0.000000,0.000000,0,0,0,0,0,29667,26694,7490,13721,0
2,2538.058105,0.000000,0.000000,0,0,0,0,0,130431,7610,7490,13721,0
3,2537.544189,0.000000,0.000000,0,0,0,0,0,116675,65463,7490,13721,0
4,2533.375488,0.000000,0.000000,0,0,0,0,0,127651,17181,7490,13721,0
5,2527.897705,0.000000,0.000000,0,0,0,0,0,69335,51818,7490,13721,0
6,2522.609863,0.000000,0.000000,0,0,0,0,0,128410,57058,7490,13721,0
7,0.000000,4321.461914,0.000000,0,0,0,0,0,78881,5487,7490,13721,0
8,0.000000,4164.022461,0.000000,0,0,0,0,0,41004,13721,7490,13721,1
9,0.000000,3976.418945,0.000000,0,0,0,0,0,78720,37921,7490,13721,0


In [4]:
scale = StandardScaler()

df.ix[:,'x1':'x8'] = scale.fit_transform(df.ix[:,'x1':'x8'])

In [6]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim
0,2.875130,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,41004,13721,7490,13721,1
1,2.651246,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,29667,26694,7490,13721,0
2,2.643647,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,130431,7610,7490,13721,0
3,2.643035,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,116675,65463,7490,13721,0
4,2.638073,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,127651,17181,7490,13721,0


In [6]:
# separating into train and test set in a stratified way
#df_train, df_val = train_test_split(df, test_size=0.5, random_state=42, stratify=df['sim'])

probes_id = np.unique(df['probe_id'].values)
train_probes, val_probes = train_test_split(probes_id, test_size=0.4, random_state=42)

df_train = df[df['probe_id'].isin(train_probes)]
df_val = df[df['probe_id'].isin(val_probes)]

print df_train.shape, df_val.shape

df_train.head()

(1591352, 13) (1060976, 13)


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim
56,2.713260,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,13945,20704,9797,13722,0
57,2.691164,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,35678,13722,9797,13722,1
58,2.664445,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,41937,15774,9797,13722,0
59,2.662103,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,10841,16034,9797,13722,0
60,2.657980,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,128491,12423,9797,13722,0


In [18]:
rbf_feature = RBFSampler(n_components=200, random_state=1, gamma=1.)
X_features = rbf_feature.fit_transform(df_train.ix[:,'x1':'x8'])


In [26]:
clf = SGDClassifier(loss='hinge', class_weight='balanced', n_jobs=12, average=10, penalty='l2', n_iter=20, alpha=1e-3, verbose=1)

clf.fit(X_features, df_train['sim'])

-- Epoch 1
Norm: 9.46, NNZs: 200, Bias: 1.096767, T: 1591352, Avg. loss: 0.170472
Total training time: 1.94 seconds.
-- Epoch 2
Norm: 9.45, NNZs: 200, Bias: 1.110240, T: 3182704, Avg. loss: 0.169332
Total training time: 3.90 seconds.
-- Epoch 3
Norm: 9.47, NNZs: 200, Bias: 1.107105, T: 4774056, Avg. loss: 0.168917
Total training time: 5.88 seconds.
-- Epoch 4
Norm: 9.46, NNZs: 200, Bias: 1.116000, T: 6365408, Avg. loss: 0.168700
Total training time: 7.80 seconds.
-- Epoch 5
Norm: 9.46, NNZs: 200, Bias: 1.116379, T: 7956760, Avg. loss: 0.168538
Total training time: 9.74 seconds.
-- Epoch 6
Norm: 9.46, NNZs: 200, Bias: 1.109863, T: 9548112, Avg. loss: 0.168455
Total training time: 11.71 seconds.
-- Epoch 7
Norm: 9.46, NNZs: 200, Bias: 1.112101, T: 11139464, Avg. loss: 0.168390
Total training time: 13.68 seconds.
-- Epoch 8
Norm: 9.46, NNZs: 200, Bias: 1.112612, T: 12730816, Avg. loss: 0.168339
Total training time: 15.67 seconds.
-- Epoch 9
Norm: 9.46, NNZs: 200, Bias: 1.110016, T: 143221

SGDClassifier(alpha=0.001, average=10, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=12,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=1, warm_start=False)

In [29]:
#df_val, _ = train_test_split(df, test_size=0.8, random_state=1, stratify=df['sim'])

X_features_val = rbf_feature.transform(df_val.ix[:,'x1':'x8'])
print clf.score(X_features_val, df_val['sim'])

df_val['predsim'] = clf.predict(X_features_val)

0.949140225604


In [30]:
df_val2 = df_val.sort_values('probe_label')
df_val2.head(40)

,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim,predsim
955132,-0.377578,-0.365324,-0.360809,-0.353817,-0.280874,-0.242941,-0.223261,2.076898,91976,44237,118413,8,0,0
955104,-0.377578,-0.365324,-0.360809,2.123347,-0.280874,-0.242941,-0.223261,-0.365169,114458,45180,118413,8,0,0
955103,-0.377578,-0.365324,-0.360809,2.198159,-0.280874,-0.242941,-0.223261,-0.365169,100017,10690,118413,8,0,0
955102,-0.377578,-0.365324,-0.360809,2.311436,-0.280874,-0.242941,-0.223261,-0.365169,93823,10690,118413,8,0,0
955101,-0.377578,-0.365324,-0.360809,4.125494,-0.280874,-0.242941,-0.223261,-0.365169,87484,8,118413,8,1,1
955100,-0.377578,-0.365324,2.082185,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,93823,10690,118413,8,0,0
955099,-0.377578,-0.365324,2.083306,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,114458,45180,118413,8,0,0
955098,-0.377578,-0.365324,2.109168,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,22881,42667,118413,8,0,0
955097,-0.377578,-0.365324,2.114814,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,71283,43758,118413,8,0,0
955096,-0.377578,-0.365324,2.188912,-0.353817,-0.280874,-0.242941,-0.223261,-0.365169,97888,47450,118413,8,0,0


In [31]:
df_val2 = df_val[df_val['predsim']==1]


accuracy = np.mean(df_val2['sim'].values == df_val2['predsim'].values)

print 'accuracy:', accuracy

accuracy: 0.927148013665


In [33]:
groups = df_val.groupby('probe_id')

# parallelization
num_cores = multiprocessing.cpu_count()
predLabel = Parallel(n_jobs=num_cores - 1)(delayed(find_label)(probe_id, group) for probe_id, group in groups)
   
predLabel = { k.items()[0][0]: k.items()[0][1] for k in predLabel }
print    
print predLabel

probes_id = np.unique(df_val['probe_id'].values.astype('int'))

probes_label = np.asarray([probes[i] for i in probes_id])
preds = np.asarray([predLabel[i] for i in probes_id])

#print preds

accuracy = np.mean(preds == probes_label)


print
print(accuracy)


{0: 31081, 1: 20928, 131077: 1131, 10: 14838, 65549: 69964, 32783: 16376, 17: 14435, 18: 40212, 149702: 37548, 137045: 38755, 141998: 11392, 120153: 25031, 32794: 27597, 32796: 25649, 32797: 28212, 65566: 17030, 65570: 29331, 32803: 38679, 36: 16317, 40: 17506, 98346: 28107, 106932: 11622, 65581: 50482, 32815: 33213, 32817: 22199, 65586: 70696, 65588: 22982, 10932: 15802, 71007: 11551, 131082: 23240, 125621: 4238, 64: 37126, 32833: 12278, 109549: 65866, 65604: 23832, 10934: 46599, 148393: 15702, 65609: 47097, 98382: 22807, 63288: 33065, 98384: 33433, 81: 46495, 65622: 17488, 32857: 16443, 32858: 30279, 131165: 42479, 32863: 13223, 65633: 52409, 65636: 7450, 32870: 1863, 32785: 31126, 153087: 26713, 81938: 6961, 65646: 34449, 129848: 42443, 98418: 37137, 10942: 9595, 131191: 31648, 118937: 65477, 121: 10169, 98427: 36501, 142942: 33207, 32894: 8212, 127: 26389, 141851: 46501, 98433: 7560, 74201: 26585, 131: 67227, 98437: 25132, 98438: 32765, 131208: 20748, 98441: 18118, 32906: 32530, 3

In [12]:

preds = np.asarray(preds)
probes_label = np.asarray(probes_label)

print preds.shape
print probes_label.shape

accuracy = np.mean(preds == probes_label)

print accuracy

(23682,)
(23682,)
0.941263406807
